### Конвертируем COCO в YOLO формат (только smoke и fire)

In [13]:
import json
from pathlib import Path

from numpy.distutils.command.config import config
from tqdm import tqdm

def convert_coco_to_yolo(coco_json_path, output_dir, target_classes):
    with open(coco_json_path) as f:
        data = json.load(f)

    # Создаем маппинг COCO ID -> YOLO class index
    class_mapping = {cid: idx for idx, cid in enumerate(target_classes)}

    # Собираем все изображения
    images = {img['id']: img for img in data['images']}

    # Группируем аннотации по image_id
    annotations = {}
    for ann in data['annotations']:
        if ann['category_id'] in target_classes:
            img_id = ann['image_id']
            if img_id not in annotations:
                annotations[img_id] = []
            annotations[img_id].append(ann)

    for img_id, anns in tqdm(annotations.items(), desc="Converting annotations"):
        img_info = images[img_id]
        original_path = Path(img_info['file_name'])

        # Создаем путь для YOLO аннотаций с сохранением структуры подпапок
        txt_path = Path(output_dir) / original_path.parent / (original_path.stem + ".txt")
        txt_path.parent.mkdir(parents=True, exist_ok=True)

        with open(txt_path, 'w') as f:
            for ann in anns:
                x, y, w, h = ann['bbox']
                image_w = img_info['width']
                image_h = img_info['height']

                x_center = (x + w/2) / image_w
                y_center = (y + h/2) / image_h
                w_norm = w / image_w
                h_norm = h / image_h

                class_id = class_mapping[ann['category_id']]
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

In [15]:
convert_coco_to_yolo(
    coco_json_path="data/474_fire_val/annotations/instances_default.json",
    output_dir="labels",
    target_classes=[4, 5]  #4=smoke, 5=fire
)

Converting annotations: 100%|██████████| 308/308 [00:00<00:00, 3433.68it/s]


### Создание data.yaml

In [22]:
yaml_content = """
train: images/train
val: images/val

names:
  0: smoke
  1: fire

augment: True
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 45
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0001
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.1
"""

In [ ]:
with open("fire_smoke.yaml", "w") as f:
    f.write(yaml_content.strip())

### Избавился от ошибки:
```
BrokenPipeError: [WinError 232] Идет закрытие канала
RuntimeError: DataLoader worker (pid(s) 27584, 10260, 20160, 3612, 3084, 380, 3856, 25208) exited unexpectedly
```


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

### Дообучаем модель

In [3]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')  # Предобученная модель

In [3]:
results = model.train(
    data='fire_smoke.yaml',
    epochs=100,
    imgsz=640,
    batch=0.7,
    lr0=0.01,
    pretrained=True,
    name='fire_smoke_detection',
    optimizer='AdamW',
    seed=42,
    plots=True,
    device=0,  # '0' для GPU
    val=True
)

New https://pypi.org/project/ultralytics/8.3.107 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=fire_smoke.yaml, epochs=100, time=None, patience=100, batch=0.7, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=fire_smoke_detection, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

train: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\train.cache... 1142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1142/1142 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 70.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3060) 12.00G total, 0.21G reserved, 0.19G allocated, 11.59G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    20054550       68.19         1.653          29.9         177.5        (1, 3, 640, 640)                    list
    20054550       136.4         2.311         26.91         108.2        (2, 3, 640, 640)                    list
    20054550       272.8         3.379         50.83         106.2        (4, 3, 640, 640)                    list
    20054550       545.5         5.604         94.72         155.2        (8, 3, 640, 640)                    list
    20054550        1091         9.685         186.6         249.6       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 14 for CUDA:0 8.65G/12.00G (72%) 


train: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\train.cache... 1142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1142/1142 [00:00<?, ?it/s]
val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]


...

    100/100      9.51G      1.237      1.026      1.312         26        640: 100%|██████████| 82/82 [00:28<00:00,  2.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.24it/s]

                   all        308       1285      0.614      0.414      0.454      0.231



100 epochs completed in 0.911 hours.
Optimizer stripped from runs\detect\fire_smoke_detection\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\fire_smoke_detection\weights\best.pt, 40.5MB

Validating runs\detect\fire_smoke_detection\weights\best.pt...
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.90it/s]


                   all        308       1285       0.59      0.415      0.454      0.233
                 smoke        250        524      0.592      0.317      0.356      0.166
                  fire        303        761      0.588      0.514      0.553        0.3
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\fire_smoke_detection


### Распределение классов
<img src="runs/detect/fire_smoke_detection/labels.jpg" alt="Компьютер" width="640" height="640">

### Результаты
<img src="runs/detect/fire_smoke_detection/results.png" alt="Компьютер" width="800" height="400">

### Берем лучшие веса и найдём оптимальный confidence threshold

In [4]:
model = YOLO('runs/detect/fire_smoke_detection/weights/best.pt')

In [11]:
import numpy as np

conf_thresholds = np.linspace(0.1, 0.9, 20)
map_scores = []

for thresh in conf_thresholds:
    metrics = model.val(conf=thresh, split='val', verbose=False)  # Отключаем подробный вывод
    map50 = metrics.box.map50
    map_scores.append(map50)
    print(f"Thresh: {thresh:.2f} | mAP50: {map50:.4f}")

# Находим лучший результат
best_idx = np.argmax(map_scores)
best_thresh = conf_thresholds[best_idx]
best_map = map_scores[best_idx]

print(f"\n{' Optimal Results ':-^40}")
print(f"| Best threshold: {best_thresh:.4f}")
print(f"| Best mAP50:     {best_map:.4f}")
print('-'*40)

Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.91it/s]


                   all        308       1285      0.591      0.415      0.494      0.271
Speed: 0.2ms preprocess, 9.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\val55
Thresh: 0.10 | mAP50: 0.4942
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:04<00:00,  4.83it/s]


                   all        308       1285      0.591      0.415      0.502       0.28
Speed: 0.2ms preprocess, 10.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\val56
Thresh: 0.14 | mAP50: 0.5015
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


...

                   all        308       1285      0.945     0.0976      0.522      0.379
Speed: 0.2ms preprocess, 9.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val72
Thresh: 0.82 | mAP50: 0.5217
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.46it/s]


                   all        308       1285      0.983      0.053      0.517      0.397
Speed: 0.4ms preprocess, 10.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val73
Thresh: 0.86 | mAP50: 0.5175
Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.76it/s]


                   all        308       1285        0.9    0.00842      0.454      0.365
Speed: 0.3ms preprocess, 9.9ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\val74
Thresh: 0.90 | mAP50: 0.4543

----------- Optimal Results ------------
| Best threshold: 0.5632
| Best mAP50:     0.5578
----------------------------------------


### Запускаем валидацию

In [12]:
metrics = model.val(
    conf=0.56,
    split='val',  # Используем тестовый набор
    device='0'
)

Ultralytics 8.3.106  Python-3.8.19 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sasha\Documents\ML_Projects\MindSet\datasets\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.03it/s]


                   all        308       1285      0.817      0.292      0.559      0.352
                 smoke        250        524       0.81      0.227      0.518      0.311
                  fire        303        761      0.824      0.357      0.599      0.393
Speed: 0.2ms preprocess, 10.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\val75


In [13]:
import pandas as pd

# Извлечение метрик по классам
class_names = model.names  # Автоматически загружаем имена классов из data.yaml

results = []
for class_id in range(len(class_names)):
    results.append({
        'Объект': class_names[class_id],
        'Precision': metrics.box.p[class_id],   # Precision для класса
        'Recall': metrics.box.r[class_id],      # Recall для класса
        'mAP50': metrics.box.maps[class_id]    # mAP50 общий (или metrics.box.maps[class_id] для per-class)
    })

df = pd.DataFrame(results)
df

,Объект,Precision,Recall,mAP50
0,smoke,0.809524,0.227099,0.310733
1,fire,0.824242,0.357424,0.392951


### Просчитаем скорость работы алгоритма

In [16]:
import time

frame = 'datasets/images/val/45a4c46c-727a-4658-a819-9571de3c3275.jpg'

# Замер скорости
n_runs = 100
start = time.perf_counter()
for _ in range(n_runs):
    _ = model.predict(frame, verbose=False)
fps = n_runs / (time.perf_counter() - start)

print(f"Скорость: {fps:.2f} FPS")

Скорость: 68.10 FPS


### Пример инференса

In [22]:
model.predict('datasets/images/val/1_23.jpg', save=True)

Results saved to runs\detect\predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'smoke', 1: 'fire'}
 obb: None
 orig_img: array([[[191, 192, 188],
         [190, 191, 187],
         [191, 192, 188],
         ...,
         [245, 223, 211],
         [245, 223, 211],
         [245, 223, 211]],
 
        [[192, 193, 189],
         [191, 192, 188],
         [191, 192, 188],
         ...,
         [245, 223, 211],
         [245, 223, 211],
         [245, 223, 211]],
 
        [[193, 194, 190],
         [192, 193, 189],
         [192, 193, 189],
         ...,
         [245, 223, 211],
         [245, 223, 211],
         [245, 223, 211]],
 
        ...,
 
        [[120, 122, 122],
         [127, 132, 131],
         [135, 140, 139],
         ...,
         [148, 180, 179],
         [145, 176, 175],
         [131, 163, 162]],
 
        [[122, 127, 126],
         [124, 129, 128],
         [133, 139, 138],
         ...,
        

<img src="runs/detect/predict2/1_23.jpg" width="800" height="600">